In [1]:
import numpy as np
import csv
import pandas
import random
from keras.datasets import imdb
from keras.models import Sequential, load_model
from keras.layers import Dense , Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import collections
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
filename1='motivation - Sheet1.csv'
filename3='happy - Sheet1.csv'
#compiled is just happy and motivation quotes in one file
filename='compiled.csv'
#can add more classes here like calm by including csv files

In [4]:
def read_csv_shuffle(filename):
    dataframe = pandas.read_csv(filename, header=None)
    ds = dataframe.sample(frac=1)
    dataset = ds.values
    X = dataset[:,0]
    Y = dataset[:,1]
    return X,Y
X,Y=read_csv_shuffle(filename)

In [5]:
def generate_y_labels(Y):
    y_train=[]
    for var in Y:
        if(var=="Happy"):
            y_train.append(1)
        else:
            y_train.append(0)
    return y_train
y_train=generate_y_labels(Y)

In [6]:
def generate_all_sentences(X):
    sentences=[]
    for row in X:
        #print(row)
        sentences.append(tf.compat.as_str(row).split())
    return sentences
sentences=generate_all_sentences(X)
#print(sentences[6]) 
#print(y_train[6])

In [7]:
def get_word_list(sentences):
    word_list=[]
    for sentence in sentences:
        for word in sentence:
            word_list.append(word)
    return word_list
#split=sentence.split()
#print(split)
word_list=get_word_list(sentences)
#print(word_list)

In [8]:
#I need to convert sentences to dictionary form
#for that I need a dict
def file_to_word_ids(word_list,sentences,word_to_id):
    new_sentences=[]
    for sentence in sentences:
        new_sentences.append([word_to_id[word] for word in sentence if word in word_to_id])
    #data = word_list
    return new_sentences

def get_dict(word_list,sentences):
    counter = collections.Counter(word_list)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    vocabulary = len(word_to_id)
    reverse_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))
    train_data = file_to_word_ids(word_list,sentences, word_to_id)
    return word_to_id,reverse_dictionary,vocabulary, train_data
dictionary,reverse_dictionary,vocabulary, X_train=get_dict(word_list,sentences)
print(len(X_train))
print(vocabulary)

230
782


In [9]:
max_review_length = 18
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)

In [10]:
X_test=X_train[192:224]
y_test=y_train[192:224]

In [11]:
X_train=X_train[:192]
#Y_train=Y_train[:160]
y_train=y_train[:192]

In [12]:
data_path='QuoteBinaryData/'

In [15]:
# create the model
def build_model(hidden_size=32,use_dropout=True):
    #embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(vocabulary, hidden_size, input_length=max_review_length))
    #model.add(LSTM(hidden_size,return_sequences=True))
    model.add(LSTM(hidden_size))
    if use_dropout:
        model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model
def train_model(model,reload_filename='NULL',save_best_only=True,batch_size=32,num_epochs=10):
    if(save_best_only):
        checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1,monitor='val_acc', save_best_only=True, mode='auto')
    else:
        checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}-{loss:.4f}.hdf5', verbose=1)
    #period option in checkpointer -> after how many epochs to save the model
    if(reload_filename!="NULL"):
            model = load_model(data_path + "/" + reload_filename)
    model.fit(X_train, y_train, validation_split=0.3, epochs=num_epochs, batch_size=batch_size,callbacks=[checkpointer])
    #model.save(data_path + "final_model.hdf5")
    #if needed save final model
hidden_size=32
rnn_model=build_model()
#add call back to save model after every epoch

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 18, 32)            25024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 33,377
Trainable params: 33,377
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
train_model(model=rnn_model,save_best_only=True,reload_filename="model-10-0.0704.hdf5", num_epochs=50)

Train on 134 samples, validate on 58 samples
Epoch 1/50
134/134 [==============================] - 2s 13ms/step - loss: 0.0542 - acc: 1.0000 - val_loss: 0.5962 - val_acc: 0.8103

Epoch 00001: val_acc improved from -inf to 0.81034, saving model to QuoteBinaryData//model-01-0.0542.hdf5
Epoch 2/50
134/134 [==============================] - 0s 716us/step - loss: 0.0490 - acc: 1.0000 - val_loss: 0.7714 - val_acc: 0.7414

Epoch 00002: val_acc did not improve from 0.81034
Epoch 3/50
134/134 [==============================] - 0s 805us/step - loss: 0.0535 - acc: 0.9925 - val_loss: 0.6095 - val_acc: 0.8276

Epoch 00003: val_acc improved from 0.81034 to 0.82759, saving model to QuoteBinaryData//model-03-0.0535.hdf5
Epoch 4/50
134/134 [==============================] - 0s 731us/step - loss: 0.0405 - acc: 1.0000 - val_loss: 0.6704 - val_acc: 0.7586

Epoch 00004: val_acc did not improve from 0.82759
Epoch 5/50
134/134 [==============================] - 0s 940us/step - loss: 0.0353 - acc: 1.0000 - va

134/134 [==============================] - 0s 694us/step - loss: 0.0044 - acc: 1.0000 - val_loss: 1.0185 - val_acc: 0.8448

Epoch 00044: val_acc did not improve from 0.84483
Epoch 45/50
134/134 [==============================] - 0s 731us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 1.0615 - val_acc: 0.7759

Epoch 00045: val_acc did not improve from 0.84483
Epoch 46/50
134/134 [==============================] - 0s 597us/step - loss: 0.0027 - acc: 1.0000 - val_loss: 1.0536 - val_acc: 0.7759

Epoch 00046: val_acc did not improve from 0.84483
Epoch 47/50
134/134 [==============================] - 0s 649us/step - loss: 0.0027 - acc: 1.0000 - val_loss: 1.0304 - val_acc: 0.8448

Epoch 00047: val_acc did not improve from 0.84483
Epoch 48/50
134/134 [==============================] - 0s 746us/step - loss: 0.0024 - acc: 1.0000 - val_loss: 1.0342 - val_acc: 0.8448

Epoch 00048: val_acc did not improve from 0.84483
Epoch 49/50
134/134 [==============================] - 0s 679us/step - loss: 0.002

In [26]:
def test_model_accuracy(filename):
    model = load_model(data_path + "/" + filename)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy: %.2f%%" % (scores[1]*100))
filename="model-10-0.0704.hdf5"
#change according to the file generated
test_model_accuracy(filename)

32/32 [==============================] - 1s 22ms/step
Accuracy: 56.25%
